In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from addict import Dict
from omegaconf import OmegaConf
from pathlib import Path

base_config = OmegaConf.load(Path("configs") / "base_config.yml")
config = OmegaConf.load(Path("configs") / "config_run_1.yml")
conf = OmegaConf.merge(base_config, config)

In [3]:
conf

{'root_dir': '/mnt/c/Users/david/Documents/Uni/vitssm', 'experiment': {'wandb': {'project': 'ViTSSM', 'group': 'mamba', 'name': 'run_1'}, 'log_freq': 50, 'checkpoint_path': 'checkpoints'}, 'dataset': {'root': 'data', 'name': 'ucf101', 'mode': 'train', 'frames_per_clip': 150, 'step_between_clips': 150, 'resolution': 64, 'train_percentage': 0.9, 'num_workers': 16, 'pin_memory': True}, 'optimization': {'loss': {'name': 'MSELoss', 'args': {}}, 'optimizer': {'name': 'AdamW', 'kwargs': {'lr': 0.001, 'weight_decay': 0.01}}, 'scheduler': {'name': 'StepLR', 'kwargs': {'step_size': 1, 'gamma': 0.5}}, 'epochs': 10, 'batch_size': 256, 'val_batch_size': 1024}, 'metrics': {'top1_accuracy': {'name': 'Accuracy', 'kwargs': {'task': 'multiclass', 'num_classes': 100, 'top_k': 1}}, 'top5_accuracy': {'name': 'Accuracy', 'kwargs': {'task': 'multiclass', 'num_classes': 100, 'top_k': 5}}}, 'model': {'device': 'cuda', 'use_amp': True}}

In [6]:
from vitssm.engine.action_recognition import ActionRecognitionEngine
import torch

model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
engine = ActionRecognitionEngine(model, conf)


Using cache found in /home/david/.cache/torch/hub/pytorch_vision_v0.10.0
 33%|███▎      | 279/833 [01:27<02:54,  3.18it/s]


KeyboardInterrupt: 

In [7]:
from vitssm.utils.metrics import MetricContainer
from torchmetrics import Accuracy, Precision, Recall
import torch

x = torch.randn(100, 100)
y = torch.randint(0, 2, (100, 100))
metric_col(x, y)

{'top1_accuracy': tensor(0.), 'top5_accuracy': tensor(0.0241)}

In [3]:
from vitssm.engine import build_optimizer
from addict import Dict
from torch import nn

config = Dict({
    "optimization": {
        "optimizer": {
            "name": "Adam",
            "args": {
                "lr": 0.001,
                "weight_decay": 0.0
            }
        }
    }    
})

model = nn.Linear(10, 10)
build_optimizer(model, config)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0.0
)

In [1]:
from vitssm.data import build_dataset
from addict import Dict
import kappaconfig as kc
import os
from pathlib import Path

config = kc.from_file_uri(Path("configs") / "base_config.yml")
print(config.keys())
res = kc.DefaultResolver()
config = Dict(res.resolve(config))

dataset = build_dataset(config)

dict_keys(['root_dir', 'experiment', 'dataset', 'optimization', 'model'])


100%|██████████| 833/833 [03:35<00:00,  3.86it/s]
/home/david/micromamba/envs/vitssm/lib/python3.12/site-packages/torchvision/datasets/video_utils.py:219: UserWarning: There aren't enough frames in the current video to get a clip for the given clip length and frames between clips. The video (and potentially others) will be skipped.
  warnings.warn(


In [6]:
dataset[1000][2]

17

In [20]:
print(os.getcwd())
os.chdir()

/mnt/c/Users/david/Desktop/ViTSSM


In [5]:
dataset

NameError: name 'dataset' is not defined

In [6]:
train_dataset[0].permute(0, 2, 3, 1).repeat(1, 1, 1, 3).shape

NameError: name 'train_dataset' is not defined

In [9]:
from torchvision.io import write_video

write_video("test.mp4", train_dataset[0].permute(0, 2, 3, 1).repeat(1, 1, 1, 3), fps=10)